In [1]:
import numpy as np
import itertools
import pandas as pd
npPath = '/sharedData2/scratch/fall_npy/'
ncPath = '/sharedData/scratch/'
acmPath = '/sharedData/scratch/all_npy3/'
DATAPATH = '/goesdata'
import re
import os
from os import path as op
import matplotlib.pyplot as plt

import xarray as xr
import netCDF4
from pathlib import Path

#refactor to create folders using subprocess library
#from subprocess import

import matplotlib.dates as mdates
%matplotlib inline

from datetime import datetime


/scratch/adomakor412/conda/envs/MyEnv/lib/python3.6/site-packages/dask/config.py:131: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
def Rad2BT(rad, planck_fk1, planck_fk2, planck_bc1, planck_bc2):
    """Radiances to Brightness Temprature (using black body equation)"""
    invRad = np.array(rad)**(-1)
    arg = (invRad*planck_fk1) + 1.0
    T = (- planck_bc1+(planck_fk2 * (np.log(arg)**(-1))) )*(1/planck_bc2) 
    return T

# April Data

In [ ]:
Tmean = []
times = []
toi = ['0300','0910','1010','1350','1510','2100']
for (bb,dd) in list(itertools.product([8],range(11))):
    for SS in ['16','17']:
        bins = np.linspace(220,250,101)
        fig, axes = plt.subplots(ncols=6, nrows =1, figsize=(15, 5), constrained_layout=True, sharey = True, sharex = True)
        fig.suptitle(f'GOES-{SS} ABI Band {bb} {8+dd}-Apr-2019', fontsize = 22, fontweight = 'bold')
        BB = str(bb).zfill(2)
        DD = str(98+dd).zfill(3)
        lookup = f'M6C{BB}_G{SS}_s2019{DD}'
        ncFiles = [f for f in os.listdir(ncPath) if re.search(lookup,f)]
        npFiles = [f for f in os.listdir(npPath) if re.search(lookup,f)]

        for ncf, npf in zip(ncFiles,npFiles):
            imageBox = np.load(op.join(npPath,npf))
            myFile = xr.open_dataset(op.join(ncPath,ncf))
            planck_fk1 = float(myFile['planck_fk1'].data)
            planck_fk2 = float(myFile['planck_fk2'].data) 
            planck_bc1 = float(myFile['planck_bc1'].data)                       
            planck_bc2 = float(myFile['planck_bc2'].data)     
            T = Rad2BT(imageBox.mean(), planck_fk1, planck_fk2, planck_bc1, planck_bc2)#unfilterd
            tString = ncf[34:38]
            times.append(tString)
            Tmean.append(T)

            if tString in toi:
                ax = axes[toi.index(tString)]
                hh = tString[:2]
                mm = tString[2:4]
                ax.set_title(f'{hh}:{mm}')
                ax.set_ylabel('density (unitless)',fontsize = 16)
                ax.set_xlabel('Temperature (K)', fontsize = 16)

                Tbox = Rad2BT(imageBox, planck_fk1, planck_fk2, planck_bc1, planck_bc2)
                ax.hist(Tbox.ravel(), bins = bins, density = True, label = 'Temp dist.')
                #ax.plot([T,T], [0,0.3],'r', label = 'something')
                ax.axvline(T, color='r',label = 'NOAA avg.')
            axes[0].legend(loc='upper left')
        fig.savefig('..'+ op.join(DATAPATH,f"Histograms_unfiltered/G_{SS}_band{BB}_04-{str(8+dd).zfill(2)}-2019"))